Most work disappeared so there you go.

In [2]:
from cStringIO import StringIO
import gzip
import os

import gcp
from gcp import storage
from gcp import bigquery as bq

In [3]:
# Import zip file from Google Cloud Storage
project = gcp.Context.default().project_id
# project = 'datalab-dot-alien-oarlock-130015'
bucket_name = project + '-datalab'
bucket_path = 'gs://' + bucket_name + '/data'
print 'Bucket: ' + bucket_path
print 'Object: ' + bucket_object

Bucket: gs://datalab-projects-1331-datalab/data
Object: gs://datalab-projects-1331-datalab/data/citydata.tar.gz


In [ ]:
del compressed_file

In [4]:
# Downloading and extracting the files from cloud storage.

def process_datafile(localpath, storagepath, table):
  gzip_filename = 'citydata.tar.gz'
  tar_filename = 'citydata.tar'
  datadir = 'season_1'
  # If data has not been extracted, extract it.
  if not os.path.isdir(datadir):
    # If citydata.tar has not been downloaded, download it
    if not os.path.isfile(tar_filename):
      
      bucket_object = '/'.join(storagepath.split('/')[:3]) + '/citydata.tar.gz'
      %storage read --object $bucket_object --variable compressed_file
      
      gzip_file = gzip.GzipFile(fileobj=StringIO(compressed_file))

      import shutil
      with open(tar_filename, 'wb') as f_out:
        shutil.copyfileobj(gzip_file, f_out)

    import tarfile
    tar = tarfile.open(tar_filename, "r")
    tar.extractall()
    tar.close()
    os.remove(tar_filename)

  # Upload extracted file into GCS
  bucketname = storagepath.split('/')[2]
  itempath = '/'.join(storagepath.split('/')[3:])
  item = storage.Item(bucketname, itempath)
  if not item.exists():
    f = open(localpath, 'rb')
    item.write_to(f.read())
    f.close()
  # Load data into Google BigQuery
  table.load(storagepath, mode='write', csv_options=gcp.CSVOptions(field_delimiter='\t'))
  

# Process Test Data

In [5]:
table = bq.Table('datalab-projects-1331:xjk_algo_comp_test.districts')
localpath = 'season_1/test_1/cluster_map/cluster_map'
storagepath = os.path.join(bucket_path,localpath)
process_datafile(localpath, storagepath, table)

NameError: global name 'compressed_file' is not defined